<a href="https://colab.research.google.com/github/pedrosousaalves/monitoramento_eletrico_grupo5/blob/main/Monitoramento_Eletrico_Grupo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Lista de Listas: Circuitos e suas medições (Nome, Tipo, V, I, FP, F, Data)
circuitos = [
    ["Circuito 1", "iluminacao", 220.0, 8.5, 0.95, 60.0, "05/11/2025"],
    ["Motor Bomba", "motor", 220.0, 14.0, 0.78, 60.0, "05/11/2025"],
]

# Tarefa 3.1: Adicionar mais 2 circuitos
circuitos.append(["Alimentador Principal", "alimentador", 220.0, 25.0, 0.92, 60.0, "05/11/2025"])
circuitos.append(["Banco Tomadas Sala 2", "tomada", 127.0, 9.5, 0.88, 60.0, "03/11/2025"])


# Dicionário: Limites de segurança por tipo de circuito
limites = {
    "iluminacao": {"i_max": 10.0, "fp_min": 0.9, "tensao_nom": 220},
    "motor": {"i_max": 20.0, "fp_min": 0.75, "tensao_nom": 220},
    "tomada": {"i_max": 15.0, "fp_min": 0.8, "tensao_nom": 127},
    "alimentador": {"i_max": 40.0, "fp_min": 0.92, "tensao_nom": 220},
}

tolerancia_tensao = 0.10  # 10% de tolerância para tensão

In [12]:
def dentro_da_faixa(circuito):
    """Verifica se um circuito está dentro dos limites de segurança."""
    nome, tipo, v, i, fp, f, data = circuito
    regra = limites.get(tipo, None)

    if not regra:
        print(f"Aviso: Tipo de circuito '{tipo}' não tem limites definidos. Assumido conforme.")
        return True

    # Validação de Tensão
    v_min = regra["tensao_nom"] * (1 - tolerancia_tensao)
    v_max = regra["tensao_nom"] * (1 + tolerancia_tensao)
    if not (v_min <= v <= v_max):
        # print(f"{nome}: Tensão ({v}V) fora da faixa ({v_min}-{v_max}V)") # Debug
        return False

    # Validação de Corrente
    if i > regra["i_max"]:
        # print(f"{nome}: Corrente ({i}A) acima do limite ({regra['i_max']}A)") # Debug
        return False

    # Validação de Fator de Potência
    if fp < regra["fp_min"]:
        # print(f"{nome}: FP ({fp}) abaixo do mínimo ({regra['fp_min']})") # Debug
        return False

    return True

def registrar_medicao(linha):
    """
    Processa uma string de medição e atualiza os dados do circuito.
    Ex: "Circuito 1; V=213; I=11.2; fp=0.82; f=60"
    """
    partes = linha.split(";")
    nome = partes[0].strip()
    medidas = {}

    # Processa as medições no formato Chave=Valor
    for pedaco in partes[1:]:
        pedaco = pedaco.strip()
        if "=" in pedaco:
            k, v = pedaco.split("=")
            medidas[k.strip().lower()] = v.strip()

    # Encontra e atualiza o circuito
    encontrado = False
    for c in circuitos:
        if c[0] == nome:
            if "v" in medidas: c[2] = float(medidas["v"])
            if "i" in medidas: c[3] = float(medidas["i"])
            if "fp" in medidas: c[4] = float(medidas["fp"])
            if "f" in medidas: c[5] = float(medidas["f"])
            # Atualiza a data (opcionalmente)
            # import datetime; c[6] = datetime.date.today().strftime("%d/%m/%Y")
            encontrado = True
            print(f"Medição de '{nome}' atualizada.")
            break

    if not encontrado:
        print(f"Erro: Circuito '{nome}' não encontrado para atualização.")

def salvar_circuitos(nome_arquivo="circuitos.txt"):
    """Salva a lista de circuitos em um arquivo TXT, separando por ponto e vírgula."""
    with open(nome_arquivo, "w") as arq:
        for c in circuitos:
            # Converte todos os elementos para string para salvar
            linha = f"{c[0]};{c[1]};{c[2]};{c[3]};{c[4]};{c[5]};{c[6]}\n"
            arq.write(linha)
    print("Circuitos salvos em", nome_arquivo)

def gerar_relatorio_nao_conforme(nome_arquivo="relatorio_nao_conforme.txt"):
    """Gera um relatório listando todos os circuitos fora da faixa segura."""
    nao_conformes = [c for c in circuitos if not dentro_da_faixa(c)]

    with open(nome_arquivo, "w") as arq:
        arq.write("RELATÓRIO DE NÃO CONFORMIDADE\n")
        arq.write(f"Data de Geração: {circuitos[0][6]}\n\n") # Usa a data do primeiro circuito

        if not nao_conformes:
            arq.write("Nenhum circuito fora da faixa segura.")

        for c in nao_conformes:
            arq.write(f"Circuito: {c[0]} (Tipo: {c[1]})\n")
            arq.write(f"  V={c[2]} V | I={c[3]} A | fp={c[4]} | f={c[5]} Hz\n")

            # Adiciona o motivo da não-conformidade (melhoria)
            nome, tipo, v, i, fp, f, data = c
            regra = limites.get(tipo)

            motivos = []
            if regra:
                v_min = regra["tensao_nom"] * (1 - tolerancia_tensao)
                v_max = regra["tensao_nom"] * (1 + tolerancia_tensao)

                if not (v_min <= v <= v_max):
                    motivos.append(f"Tensão ({v}V) fora da faixa ({v_min:.0f}-{v_max:.0f}V).")
                if i > regra["i_max"]:
                    motivos.append(f"Corrente ({i}A) acima do limite ({regra['i_max']}A).")
                if fp < regra["fp_min"]:
                    motivos.append(f"Fator de Potência ({fp}) abaixo do mínimo ({regra['fp_min']}).")

            arq.write(f"  Motivo(s): {', '.join(motivos)}\n\n")

    print("Relatório de Não Conformidade gerado em", nome_arquivo)

def resumo_eletrico():
    """Realiza análises elétricas e exibe um resumo."""

    if not circuitos:
        print("Atenção: Não há circuitos para analisar.")
        return

    # 1. Menor Fator de Potência (fp)
    menor_fp = min(circuitos, key=lambda x: x[4])

    # 2. Circuitos Fora da Faixa
    fora = [c for c in circuitos if not dentro_da_faixa(c)]

    # 3. Cálculo de Sobrecarga (usando percentual em relação ao I_max)
    sobrecarga = []
    for c in circuitos:
        nome, tipo, v, i, fp, f, data = c
        regra = limites.get(tipo)
        if regra and i > 0:
            percentual = (i / regra["i_max"]) * 100
            sobrecarga.append((nome, percentual))

    # Circuito mais sobrecarregado (se houver dados de sobrecarga)
    mais_sobrecarregado = max(sobrecarga, key=lambda x: x[1]) if sobrecarga else ("N/D", 0)

    print("\n=== RESUMO ELÉTRICO ===")
    print(f"* Total de Circuitos: {len(circuitos)}")
    print(f"* Total Fora da Faixa: {len(fora)}")
    print(f"* Circuito com Menor FP: {menor_fp[0]} ({menor_fp[4]:.2f})")
    print(f"* Circuito Mais Sobrec.: {mais_sobrecarregado[0]} ({mais_sobrecarregado[1]:.1f} % do I_max)")

In [13]:
def simular_arquivo_uso(nome_arquivo="uso_diario.txt"):
    """
    Cria um arquivo de simulação para o módulo extra.
    Formato: Circuito;Tempo_Ligado_Horas;Tarifa_Custo_R$/kWh
    """
    simulacao = [
        "Circuito 1; 8.0; 0.75",
        "Motor Bomba; 12.0; 0.75",
        "Alimentador Principal; 24.0; 0.75",
        "Banco Tomadas Sala 2; 4.0; 0.75",
    ]
    with open(nome_arquivo, "w") as arq:
        arq.write("\n".join(simulacao))
    print(f"Arquivo de simulação '{nome_arquivo}' criado.")

def ler_dados_uso(nome_arquivo="uso_diario.txt"):
    """
    Lê o arquivo de dados de uso diário.
    Retorna um dicionário: {Nome_Circuito: [Tempo_Horas, Tarifa_R$/kWh]}
    """
    dados_uso = {}
    try:
        with open(nome_arquivo, "r") as arq:
            for linha in arq:
                if not linha.strip(): continue

                try:
                    partes = [p.strip() for p in linha.split(";")]
                    if len(partes) == 3:
                        nome = partes[0]
                        tempo = float(partes[1])
                        tarifa = float(partes[2])
                        dados_uso[nome] = [tempo, tarifa]
                except ValueError:
                    print(f"Aviso: Linha inválida no arquivo de uso: {linha.strip()}")
    except FileNotFoundError:
        print(f"ERRO: Arquivo '{nome_arquivo}' não encontrado. Use a função simular_arquivo_uso().")
    return dados_uso

def modulo_extra():
    """Calcula o consumo de energia (kWh) e o custo (R$) dos circuitos."""
    print("\n--- RODANDO MÓDULO DE CONSUMO DE ENERGIA ---")

    # Garante que o arquivo de dados de uso existe para o teste
    simular_arquivo_uso()
    dados_uso = ler_dados_uso()

    if not dados_uso:
        return

    relatorio = []
    relatorio.append(f"{'Circuito':<20}{'V (V)':>5}{'I (A)':>8}{'FP':>5}{'P (kW)':>8}{'Tempo (h)':>10}{'Consumo (kWh)':>15}{'Custo (R$)':>12}")
    relatorio.append("-" * 90)

    custo_total = 0.0

    for c in circuitos:
        nome, tipo, v, i, fp, f, data = c

        if nome in dados_uso:
            tempo_horas, tarifa = dados_uso[nome]

            # Cálculo da Potência Ativa (P = V * I * FP)
            potencia_w = v * i * fp
            potencia_kw = potencia_w / 1000.0

            # Cálculo do Consumo (kWh) e Custo
            consumo_kwh = potencia_kw * tempo_horas
            custo_rs = consumo_kwh * tarifa
            custo_total += custo_rs

            linha = f"{nome:<20}{v:>5.0f}{i:>8.2f}{fp:>5.2f}{potencia_kw:>8.3f}{tempo_horas:>10.1f}{consumo_kwh:>15.3f}{custo_rs:>12.2f}"
            relatorio.append(linha)

    relatorio.append("-" * 90)
    relatorio.append(f"{'Custo Total Diário (R$):':<75}{custo_total:>14.2f}")

    print("\n".join(relatorio))
    print("\nRelatório de consumo diário concluído.")

In [ ]:
def main():
    """Menu principal para rodar o sistema completo."""
    while True:
        print("\n=== Sistema de Monitoramento Elétrico ===")
        print("1 - Registrar nova medição (String)")
        print("2 - Salvar circuitos em arquivo")
        print("3 - Gerar relatório de não conformidade")
        print("4 - Resumo elétrico (FP, Sobrecarga, Fora Faixa)")
        print("5 - Rodar MÓDULO EXTRA (Consumo e Custo)")
        print("0 - Sair")

        opc = input("Escolha uma opção: ")

        if opc == "1":
            # Exemplo de entrada: Circuito 1; V=213; I=11.2; fp=0.82; f=60
            linha = input("Digite a medição (Ex: Nome; V=...; I=...; fp=...; f=...)\n> ")
            registrar_medicao(linha)
        elif opc == "2":
            salvar_circuitos()
        elif opc == "3":
            gerar_relatorio_nao_conforme()
        elif opc == "4":
            resumo_eletrico()
        elif opc == "5":
            modulo_extra()
        elif opc == "0":
            print("Sistema encerrado. Não se esqueça de usar o git push!")
            break
        else:
            print("Opção inválida. Tente novamente.")

# Roda o programa
if __name__ == "__main__":
    main()


=== Sistema de Monitoramento Elétrico ===
1 - Registrar nova medição (String)
2 - Salvar circuitos em arquivo
3 - Gerar relatório de não conformidade
4 - Resumo elétrico (FP, Sobrecarga, Fora Faixa)
5 - Rodar MÓDULO EXTRA (Consumo e Custo)
0 - Sair
